# 데이터 확인

In [27]:
import os

base_dir = '../data'

# 1. action 디렉토리 내 json 파일 이름 (확장자 제거)
action_json_names = set()
action_path = os.path.join(base_dir, 'action')

for cls in os.listdir(action_path):
    cls_path = os.path.join(action_path, cls)
    if not os.path.isdir(cls_path): continue
    for session in os.listdir(cls_path):
        session_path = os.path.join(cls_path, session)
        if not os.path.isdir(session_path): continue
        for fname in os.listdir(session_path):
            if fname.endswith('.json'):
                json_name = os.path.splitext(fname)[0]
                action_json_names.add(json_name)

print(f"[ACTION] JSON 파일 수: {len(action_json_names)}")
# 유니크한 데이터(동영상) : 829개

[ACTION] JSON 파일 수: 829


In [28]:
import os
import json
from collections import Counter

# label_aliases (주어진 대로)
label_aliases = {
    "action": {
        "허리를 아치로 세움": "허리를 아치로 세우는 동작",
    },
    "situation": {
        "보호자와 떨어질 때/혼자 남겨지거나 낯선장소에 있을 때": "혼자 남겨진 상황",
        "낯선 소리가 나거나 낯선 사람을 봤을 때": "낯선 존재를 만난 상황",
        "다른 사람이나 동물을 만났을 때": "낯선 존재를 만난 상황",
        "다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때": "낯선 존재를 만난 상황",
        "낯선 소리가 났을 때": "낯선 장소/소리 상황",
        "낯선 장소에 있거나 낯선 소리가 날 때": "낯선 장소/소리 상황",
        "잠들기 전이나 같이 누워있을 때": "휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)",
    }
}

def normalize_label(category, label, alias_map):
    """alias 매핑된 라벨로 변환, 없으면 원본 그대로 반환"""
    if category in alias_map and label in alias_map[category]:
        return alias_map[category][label]
    return label

base_dir = '../data'
action_path = os.path.join(base_dir, 'action')

# 라벨별 카운터 초기화
action_counter = Counter()
emotion_counter = Counter()
situation_counter = Counter()

for cls in os.listdir(action_path):
    cls_path = os.path.join(action_path, cls)
    if not os.path.isdir(cls_path):
        continue
    for session in os.listdir(cls_path):
        session_path = os.path.join(cls_path, session)
        if not os.path.isdir(session_path):
            continue
        
        # JSON 파일 로드 (폴더 내 1개 assumed)
        json_files = [f for f in os.listdir(session_path) if f.endswith('.json')]
        if not json_files:
            continue
        json_path = os.path.join(session_path, json_files[0])
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        meta = data.get("metadata", {})
        inspect = meta.get("inspect", {})
        owner = meta.get("owner", {})

        raw_action = inspect.get("action") or meta.get("action")
        raw_emotion = inspect.get("emotion") or owner.get("emotion")
        raw_situation = owner.get("situation")

        # 라벨 정규화
        action = normalize_label('action', raw_action, label_aliases)
        emotion = normalize_label('emotion', raw_emotion, label_aliases)
        situation = normalize_label('situation', raw_situation, label_aliases)

        # 카운터 업데이트
        if action: action_counter[action] += 1
        if emotion: emotion_counter[emotion] += 1
        if situation: situation_counter[situation] += 1

print("=== Action 라벨 분포 ===")
for label, count in action_counter.most_common():
    print(f"{label}: {count}")

print("\n=== Emotion 라벨 분포 ===")
for label, count in emotion_counter.most_common():
    print(f"{label}: {count}")

print("\n=== Situation 라벨 분포 ===")
for label, count in situation_counter.most_common():
    print(f"{label}: {count}")


=== Action 라벨 분포 ===
그루밍하는 동작: 187
허리를 아치로 세우는 동작: 180
꼬리를 흔드는 동작: 49
앞발을 뻗어 휘적거리는 동작: 48
걷거나 달리는 동작: 48
납작 엎드리는 동작: 47
배를 보여주는 동작: 46
옆으로 눕는 동작: 46
좌우로 뒹구는 동작: 45
머리를 들이대는 동작: 45
앞발로 꾹꾹 누르는 동작: 44
발을 숨기고 웅크리고 앉는 동작: 44

=== Emotion 라벨 분포 ===
편안/안정: 586
행복/즐거움: 124
공격성: 51
화남/불쾌: 30
불안/슬픔: 21
공포: 17

=== Situation 라벨 분포 ===
휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등): 286
기타: 209
먹을것, 장난감이 앞에 있을 때: 148
편안히 쓰다듬어 줄 때: 65
보호자가 집에 돌아왔을 때: 49
낯선 존재를 만난 상황: 16
낯선 장소/소리 상황: 15
빗질/발톱깍기/목욕 등 위생관리를 할 때: 9
싫어하는 부위를 만질 때: 9
산책이나 노즈워크 중: 9
밥그릇, 장난감과 같은 소유물을 만질 때: 8
산책 준비 또는 산책중일 때: 5
혼자 남겨진 상황: 1


In [ ]:
import os
import json
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

# 1) alias 매핑 정의
label_aliases = {
    "action": {
        "허리를 아치로 세움": "허리를 아치로 세우는 동작",
    },
    "situation": {
        "보호자와 떨어질 때/혼자 남겨지거나 낯선장소에 있을 때": "혼자 남겨진 상황",
        "낯선 소리가 나거나 낯선 사람을 봤을 때": "낯선 존재를 만난 상황",
        "다른 사람이나 동물을 만났을 때": "낯선 존재를 만난 상황",
        "다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때": "낯선 존재를 만난 상황",
        "낯선 소리가 났을 때": "낯선 장소/소리 상황",
        "낯선 장소에 있거나 낯선 소리가 날 때": "낯선 장소/소리 상황",
        "잠들기 전이나 같이 누워있을 때": "휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)",
    }
}

# 2) 라벨 정규화 함수
def normalize_label(category, label, label_aliases):
    if category in label_aliases and label in label_aliases[category]:
        return label_aliases[category][label]
    return label

# 3) config로부터 라벨 인덱스 생성 함수
def get_label_maps_from_config(config, label_aliases=None):
    label_aliases = label_aliases or {}

    label_maps = {}
    for category in ['action', 'emotion', 'situation']:
        raw_labels = config['label_names'][category]
        label_maps[category] = {label: i for i, label in enumerate(raw_labels)}

        # alias 매핑도 인덱스에 포함
        for alias, original in label_aliases.get(category, {}).items():
            if original not in label_maps[category]:
                raise ValueError(f"Original label '{original}' not found in {category} labels.")
            label_maps[category][alias] = label_maps[category][original]

    return label_maps

# 4) Dataset 클래스
class VideoDataset(Dataset):
    def __init__(self, video_dirs, label_encoder, label_aliases=None,
                 transform=None):
        self.video_dirs = video_dirs
        self.label_encoder = label_encoder
        self.label_aliases = label_aliases or {}
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.video_dirs)

    def __getitem__(self, idx):
        video_path = self.video_dirs[idx]

        # JSON 파일 로드
        json_files = [f for f in os.listdir(video_path) if f.endswith(".json")]
        if not json_files:
            raise FileNotFoundError(f"No JSON file found in {video_path}")
        with open(os.path.join(video_path, json_files[0]), "r", encoding="utf-8") as f:
            data = json.load(f)

        meta = data.get("metadata", {})
        inspect = meta.get("inspect", {})
        owner = meta.get("owner", {})

        # 원본 라벨 추출
        raw_action = inspect.get("action") or meta.get("action")
        raw_emotion = inspect.get("emotion") or owner.get("emotion")
        raw_situation = owner.get("situation")

        if raw_action is None or raw_emotion is None or raw_situation is None:
            raise ValueError(f"Missing label in metadata for {video_path}")

        # 라벨 정규화
        norm_action = normalize_label('action', raw_action, self.label_aliases)
        norm_emotion = normalize_label('emotion', raw_emotion, self.label_aliases)
        norm_situation = normalize_label('situation', raw_situation, self.label_aliases)

        # 인덱스 변환
        action_idx = self.label_encoder['action'][norm_action]
        emotion_idx = self.label_encoder['emotion'][norm_emotion]
        situation_idx = self.label_encoder['situation'][norm_situation]

        # 프레임 이미지 전체 불러오기
        frame_files = sorted([f for f in os.listdir(video_path) if f.endswith(".jpg")])
        frames = []
        for fname in frame_files:
            img_path = os.path.join(video_path, fname)
            img = Image.open(img_path).convert("RGB")
            img = self.transform(img)
            frames.append(img)

        if not frames:
            raise ValueError(f"No image frames found in {video_path}")

        frames_tensor = torch.stack(frames)  # (T, C, H, W)
        labels_tensor = torch.tensor([action_idx, emotion_idx, situation_idx], dtype=torch.long)

        return frames_tensor, labels_tensor
    
def custom_collate(batch):
    """
    샘플마다 프레임 수가 달라짐..
    """
    videos, labels = zip(*batch)
    return list(videos), torch.stack(labels)


In [14]:
import yaml

with open("./configs/base.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

label_encoder = get_label_maps_from_config(config, label_aliases)

import os
# 1. JSON 파일이 있는 폴더만 모으기
base_path = "../data/action"
video_dirs_with_json = []

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith('.json'):
            video_dirs_with_json.append(root)
            break

print(f"Found {len(video_dirs_with_json)} video folders with JSON")

# 2. Dataset 생성
dataset = VideoDataset(video_dirs_with_json, label_encoder, label_aliases)

# 3. DataLoader
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=custom_collate)

for videos, labels in dataloader:
    print(f"# Samples: {len(videos)}")
    print(f"1st sample shape: {videos[0].shape}")  # e.g., (T, C, H, W)
    print(f"Labels shape: {labels.shape}")         # (B, 3)
    break



Found 829 video folders with JSON
# Samples: 4
1st sample shape: torch.Size([89, 3, 224, 224])
Labels shape: torch.Size([4, 3])


In [18]:
from collections import Counter, defaultdict

def compute_label_distribution(dataset):
    action_counter = Counter()
    emotion_counter = Counter()
    situation_counter = Counter()

    for i in range(len(dataset)):
        _, label_tensor = dataset[i]
        action, emotion, situation = label_tensor.tolist()
        print(action, emotion, situation)
        action_counter[action] += 1
        emotion_counter[emotion] += 1
        situation_counter[situation] += 1

    return {
        "action": action_counter,
        "emotion": emotion_counter,
        "situation": situation_counter
    }

distribution = compute_label_distribution(dataset)

6 1 12
6 1 12
6 1 12
6 1 12
6 1 11
6 1 11
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 3
6 1 12
6 1 12
6 1 12
6 1 12
6 1 3
6 1 12
6 1 12
6 1 3
6 1 12
6 1 12
6 1 12
6 1 12
6 1 3
6 1 10
6 1 12
6 1 3
6 1 12
6 1 12
6 1 12
6 1 3
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 3
6 1 10
6 1 10
6 1 3
6 1 12
6 1 14
6 1 12
6 1 12
6 1 8
6 1 10
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 14
6 1 12
6 1 3
6 1 12
6 1 12
6 1 12
6 1 12
6 1 3
6 1 12
6 1 12
6 5 14
6 1 10
6 1 14
6 1 3
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 3
6 1 10
6 1 12
6 1 12
6 1 12
6 0 14
6 1 3
6 1 3
6 1 3
6 1 3
6 1 3
6 1 12
6 1 12
6 1 12
6 1 12
6 1 3
6 1 12
6 1 12
6 1 12
6 1 10
6 1 12
6 1 12
6 1 10
6 0 14
6 1 10
6 1 3
6 1 10
6 1 12
6 1 3
6 1 3
6 1 12
6 1 3
6 1 12
6 1 12
6 1 3
6 1 12
6 1 10
6 1 3
6 1 12
6 1 12
6 1 9
6 1 12
6 1 3
6 1 12
6 1 12
6 1 10
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 3
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 4
6 1 12
6 1 11
6 1 12
6 1 3
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 12
6 1 3
6 1 12
6 

KeyboardInterrupt: 

In [ ]:
for category, counter in distribution.items():
    print(f"\n📊 {category.upper()} label distribution:")
    for label_idx, count in sorted(counter.items()):
        label_name = list(label_encoder[category].keys())[list(label_encoder[category].values()).index(label_idx)]
        print(f"  [{label_idx:2}] {label_name:<40} : {count}")

In [25]:
from sklearn.model_selection import train_test_split
import os
import json
import glob

def extract_situation_label(video_path, label_encoder, label_aliases):
    """situation 라벨 인덱스를 추출하는 함수"""
    json_files = [f for f in os.listdir(video_path) if f.endswith(".json")]
    if not json_files:
        return None

    with open(os.path.join(video_path, json_files[0]), "r", encoding="utf-8") as f:
        data = json.load(f)

    meta = data.get("metadata", {})
    owner = meta.get("owner", {})
    raw_situation = owner.get("situation")

    if raw_situation is None:
        return None

    def normalize(category, label):
        if category in label_aliases and label in label_aliases[category]:
            return label_aliases[category][label]
        return label

    try:
        situation = label_encoder["situation"][normalize("situation", raw_situation)]
        return situation
    except:
        return None


In [26]:
# 모든 비디오 디렉토리 수집
video_dirs = glob.glob("../data/action/**/*/", recursive=True)
video_dirs = [d for d in video_dirs if os.path.isdir(d)]

# situation 인덱스 수집
X, y = [], []

for d in video_dirs:
    sit = extract_situation_label(d, label_encoder, label_aliases)
    if sit is not None:
        X.append(d)
        y.append(sit)

# stratified split
train_dirs, val_dirs = train_test_split(
    X, test_size=0.2, stratify=y, random_state=42
)

print(f"[SPLIT DONE] Train: {len(train_dirs)} / Val: {len(val_dirs)}")


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
print("Train 조합 분포:")
print(Counter([extract_label_triplet(d, label_encoder, label_aliases) for d in train_dirs]))

print("Val 조합 분포:")
print(Counter([extract_label_triplet(d, label_encoder, label_aliases) for d in val_dirs]))


Train action dist: Counter({6: 150, 0: 144, 11: 39, 12: 38, 1: 38, 5: 38, 10: 37, 7: 37, 4: 36, 2: 36, 3: 35, 9: 35})
Val action dist:   Counter({6: 37, 0: 36, 11: 10, 1: 10, 12: 10, 5: 9, 9: 9, 4: 9, 2: 9, 10: 9, 3: 9, 7: 9})
